In [12]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained MobileNetV2 model without top (include_top=False)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of MobileNetV2
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Load an image and preprocess it
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    return img_array

# Function to classify an image
def classify_image(image_path, min_line_length):
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)[0][0]
    
    # Process the image to detect lines
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=min_line_length, maxLineGap=10)
    
    # Check if there are lines and the prediction is above threshold
    # if lines is not None and prediction >= 0.5:
    print(prediction)
    if prediction >= 0.5:
        return True
    else:
        return False

image_path = './Data/fits_filtered8/tic4.png'
min_line_length = 1  # Adjust this value as needed
contains_line = classify_image(image_path, min_line_length)
if contains_line:
    print("The image contains a line.")
else:
    print("The image does not contain a line or the line is too short.")


1/1 [==============================] - 1s 1s/step
0.5022025
The image contains a line.
